## Deliverable 2. Create a Customer Travels Desinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import gmaps
import requests
import os
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
g_baseurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Mataura,NZ,-46.1927,168.8643,51.48,81,41,5.32,scattered clouds
1,1,Bluff,NZ,-46.6000,168.3333,51.80,81,25,7.16,scattered clouds
2,2,Yining,CN,29.0361,114.5636,72.63,78,0,1.81,light rain
3,3,Bethel,US,41.3712,-73.4140,65.91,92,90,3.00,mist
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,30.20,86,75,10.36,light snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
12,12,Kavieng,PG,-2.5744,150.7967,80.92,74,100,1.90,overcast clouds
15,15,Alofi,NU,-19.0595,-169.9187,78.69,78,75,12.66,broken clouds
17,17,Yanam,IN,16.7333,82.2167,77.85,89,99,1.95,overcast clouds
22,22,Odweyne,SO,9.4092,45.0640,78.69,56,99,7.67,light rain
23,23,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,75,11.50,broken clouds
31,31,Rapid Valley,US,44.0625,-103.1463,87.85,13,1,1.99,clear sky
40,40,Acapulco,MX,16.8634,-99.8901,85.82,74,40,9.22,scattered clouds
43,43,Lazaro Cardenas,MX,17.9583,-102.2000,86.29,66,37,6.04,scattered clouds
46,46,Hithadhoo,MV,-0.6000,73.0833,83.61,75,12,19.39,few clouds
48,48,Manta,EC,-0.9500,-80.7333,78.58,69,20,14.97,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        258
City           258
Country        257
Lat            258
Lng            258
Max Temp       258
Humidity       258
Cloudiness     258
Wind Speed     258
Description    258
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))

257


In [8]:
# Show new clean dataframe
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
12,12,Kavieng,PG,-2.5744,150.7967,80.92,74,100,1.90,overcast clouds
15,15,Alofi,NU,-19.0595,-169.9187,78.69,78,75,12.66,broken clouds
17,17,Yanam,IN,16.7333,82.2167,77.85,89,99,1.95,overcast clouds
22,22,Odweyne,SO,9.4092,45.0640,78.69,56,99,7.67,light rain
23,23,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,75,11.50,broken clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
12,Kavieng,PG,80.92,-2.5744,150.7967,overcast clouds,
15,Alofi,NU,78.69,-19.0595,-169.9187,broken clouds,
17,Yanam,IN,77.85,16.7333,82.2167,overcast clouds,
22,Odweyne,SO,78.69,9.4092,45.0640,light rain,
23,Arraial Do Cabo,BR,75.16,-22.9661,-42.0278,broken clouds,
31,Rapid Valley,US,87.85,44.0625,-103.1463,clear sky,
40,Acapulco,MX,85.82,16.8634,-99.8901,scattered clouds,
43,Lazaro Cardenas,MX,86.29,17.9583,-102.2000,scattered clouds,
46,Hithadhoo,MV,83.61,-0.6000,73.0833,few clouds,
48,Manta,EC,78.58,-0.9500,-80.7333,few clouds,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    g_baseurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(g_baseurl, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
12,Kavieng,PG,80.92,-2.5744,150.7967,overcast clouds,Nusa Island Retreat
15,Alofi,NU,78.69,-19.0595,-169.9187,broken clouds,Taloa Heights
17,Yanam,IN,77.85,16.7333,82.2167,overcast clouds,The Regency Hotel
22,Odweyne,SO,78.69,9.4092,45.0640,light rain,Eng Hiirey House
23,Arraial Do Cabo,BR,75.16,-22.9661,-42.0278,broken clouds,Pousada Porto Praia
31,Rapid Valley,US,87.85,44.0625,-103.1463,clear sky,Fairfield Inn & Suites by Marriott Rapid City
40,Acapulco,MX,85.82,16.8634,-99.8901,scattered clouds,HS Hotsson Smart Acapulco
43,Lazaro Cardenas,MX,86.29,17.9583,-102.2000,scattered clouds,quinta antigua
46,Hithadhoo,MV,83.61,-0.6000,73.0833,few clouds,Scoop Guest House
48,Manta,EC,78.58,-0.9500,-80.7333,few clouds,Balandra Hotel


In [13]:
len(clean_hotel_df)

238

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))